In [ ]:
library(Seurat)
library(dplyr)
library(tidyverse)
library(ggplot2)
library(MetBrewer)
library(pheatmap)
library(RColorBrewer)

In [ ]:
##Read in data
seurat <- readRDS('seurat.rds')


In [ ]:
#Read in morphogen metadata
library(readxl)
morphogen_info <- read_excel('Parse_samples_all.xlsx')
test <- as.data.frame(morphogen_info)
rownames(test) <- test$Sample
rownames(morphogen_info) <- morphogen_info$Sample

In [9]:
#Generate dataframe for %final_clustering
#Use the seurat table function
df_cl <- prop.table(table(seurat$parse_id, seurat$final_clustering), margin=1)


In [ ]:
#Read in annotation
sum.iglut.pre <- read_excel('cluster_summary.xlsx')


In [15]:
#Changing sum.iglut.post cluster metadata to dataframe
sum.iglut.pre <- as.data.frame(sum.iglut.pre)
typeof(sum.iglut.pre)
rownames(sum.iglut.pre) <- sum.iglut.pre$Cluster
head(sum.iglut.pre)


[1] "list"

,Cluster,Division,Neuron_type,Region,Score,Subtype,Subregion,Division_old,Neuron_type_old,Region_old,...,Regev_annotation_pct.Count,Regev_Type_label,Regev_Type_Mean,Regev_Type_pct.Count,Regev_Region_label,Regev_Region_Mean,Regev_Region_pct.Count,Renthal_subtype_label,Renthal_subtype_Mean,Renthal_subtype_pct.Count
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
0,0,CNS,RGL,Hindbrain,0.7656497,NA,NA,CNS,Radial glia,Hndbrain,...,80.95238,nan,0.9771262,100.00000,colon,0.8361709,64.28571,NF1,0.4879705,52.38095
1,1,CNS,RGL,Forebrain,0.8883681,NA,Thalamus,CNS,Radial glia,Forebrain,...,95.55556,nan,0.9744816,100.00000,colon,0.8411144,97.77778,NF2,0.5609552,80.00000
2,2,CNS,GLUT,Hindbrain,0.8883681,SST,NA,Mix,Sensory neuron,Hindbrain,...,100.00000,nan,0.8186653,97.82609,nan,0.8622933,76.08696,NF1,0.7398487,100.00000
3,3,CNS,NBL,Hindbrain,0.9179493,NA,Pons,CNS,Neuroblast,Hindbrain,...,90.32258,nan,0.9358544,100.00000,ileum,0.7511190,83.87097,NF1,0.6652105,64.51613
4,4,CNS,GLUT,Spinal cord,0.7517467,dl3,NA,CNS,??Neuron,Hindbrain/SC/TG,...,32.72727,nan,0.7744533,89.09091,colon,0.7721304,58.18182,NF2,0.7456172,100.00000
5,5,CNS,GLUT,Hindbrain,0.8593211,SST,Medulla,CNS,Sensory neuron,Hindbrain,...,100.00000,nan,0.7461213,74.41860,nan,0.8525731,90.69767,NF1,0.7701733,100.00000


In [30]:
my_colour = list(
    AP_axis = c(XAV_3 = "#9a133d", XAV_2 = "#d8527c", XAV_1 = "#f9b4c9", ctrl='grey90',
               CHIR_1 = "#dec5da", CHIR_2 = "#b695bc", CHIR_3 = "#90719f", CHIR_4 = "#574571",
               RA_4 = "#376795", RA_3 = "#528fad", RA_2 = "#72bcd5", RA_1 = "#aadce0",
               FGF8_1 = "plum1", FGF8_2 = "orchid2", FGF8_3 = "mediumorchid3", FGF8_4 = "magenta4",
               RA_4_CHIR = "#3c7c3d", RA_3_CHIR = "#669d62", 
               RA_2_CHIR = "#9cc184", RA_1_CHIR = "#c2d6a4",
               FGF8_1_CHIR = "gray80", FGF8_2_CHIR = "gray60",
               FGF8_3_CHIR = "gray40", FGF8_4_CHIR = "gray20",
               XAV_3_FGF8 = "#d8527c", RA_3_FGF8 = "#183E6B", CHIR_2_FGF8 = "#90719f"),
    
    DV_axis = c(BMP4_3 = "#ffb242", BMP4_2 = "#ffd353", BMP4_1 = '#ffe6b7',
               ctrl = 'grey90', SHH_1 = '#C2D9F7', SHH_2 = "#98C1F0",
               SHH_3 = "#4782DD", SHH_4 = "#1D52A1", CycA = "#669d62"),
    
    Division = c(CNS = '#a40000', PNS = '#16317d', Other='#cccccc'),
    
    Region = c(Forebrain='#fe9b00', Midbrain='#f4c40f', Hindbrain='#d8443c', `Spinal cord`= '#9b3441',
               SYM='#268a8a', ENS='#226a99', TG='#383b81', DRG='#92c051',
               `Optic cup`='#633372', NE = '#e87b89', Other='#cccccc'),
    
    Neuron_type = c(GLUT='#fe9b00', GABA='#f4c40f', CHO='#33A02C', NOR='#383b81', GLY='#d8443c', DOPA='#9b3441',
                   RGL='#6a3d9a', NBL='#c9bba2', IP='#aab5d5', NCC='#92c051', EPL='#000000',
                   EC='#99610a', MES='#cccccc', OB='#1f6e9c', EPC='#B9CECB'),
    
    Basal_media = c(NIM = '#4c825d', N2B27_2Si = '#8dc7dc', N2B27_SB_CHIR = '#0e2a4d')
)

In [ ]:
#Setting the order of all region
region <- c('Forebrain', 'Midbrain', 'Hindbrain', 'Spinal cord',
           'SYM', 'ENS', 'TG', 'DRG',
           'Optic cup', 'NE', 'Other')

In [ ]:
col_region_test7 <- c('#fe9b00', '#f4c40f', '#d8443c', '#9b3441',
               '#268a8a', '#226a99', '#383b81', '#92c051',
               '#633372', '#e87b89', '#cccccc')

In [ ]:
#Setting the identity of all neuron types
neuron_type <- c('GLUT','GABA','CHO','NOR','GLY','DOPA',
                'RGL','NBL','IP','NCC','EPL',
                'EC','MES','OB','EPC')

In [ ]:
col_neuron_type2 <- c('#fe9b00','#f4c40f','#33A02C','#383b81','#d8443c','#9b3441',
                    '#6a3d9a','#c9bba2','#aab5d5','#92c051','#000000',
                    '#99610a','#cccccc','#1f6e9c','#B9CECB')

In [ ]:
col_red <- c("#FFF5F0", "#FEE0D2", "#FCBBA1", "#FC9272", "#FB6A4A",
             "#EF3B2C", "#CB181D", "#A50F15", "#67000D", "#67000D",
            "#67000D", "#67000D")

In [ ]:
options(repr.plot.width=10, repr.plot.height=9)
pheatmap(df_cl, scale='column',
         annotation_row=test[,2:4],
         annotation_col=sum.iglut.pre[,2:4],
         annotation_colors = my_colour, 
         breaks = seq(0, 5, length.out = 50),
         cutree_rows = 17,
         cutree_cols = 11,
         color = colorRampPalette(col_red)(50),
         show_rownames = F, show_colnames = F)

In [ ]:
# Export heatmap
pheatmap(df_cl, scale='column',
         annotation_row=test[,2:4],
         annotation_col=sum.iglut.pre[,2:4],
         annotation_colors = my_colour, 
         breaks = seq(0, 5, length.out = 50),
         cutree_rows = 17,
         cutree_cols = 11,
         color = colorRampPalette(col_red)(50),
         show_rownames = F, show_colnames = F,
         border_color = NA,
         filename='Composition_heatmap.pdf',
         width=12, height=10)